In [175]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from split import *
from score import *
from scipy import interpolate
import time 
%matplotlib inline 
import warnings 
warnings.simplefilter('ignore', np.RankWarning)

In [176]:
def rho(x,y):
    return np.sqrt(x*x + y*y)
def r(x,y,z):
    return np.sqrt(x*x + y*y + z*z)
def theta(x,y,z):
    return np.arccos(z/r(x,y,z))
def phi(x,y):
    return np.arctan(y/x)
def module(r):
    return np.sqrt(np.sum(r*r))
def r_e(z, r_l, r_c):
    z_c = r_c[2] 
    r_versor = (r_l - r_c)/module(r_l - r_c)               # computing r_versor
    r_versor_dot_z_versor = r_versor[2]  
    return r_c - r_versor/r_versor_dot_z_versor*(z_c - z)  # IMPORTANT WITH THE MINUS SIGN. 

In [177]:
def reading_data(fraction):
    name = 'data/RAMPData55microns50psInner200microns50psOuter_train.txt' # To be modified for others. 
    data_fraction = fraction
    
    df = pd.DataFrame()
    df = pd.read_csv(name, sep=' ') # All data.
    df,_ = Split_frac(df, data_fraction)
    return df

In [178]:
# DATAFRAME will be a global data. 
fraction = 0.004 
dphi     = 0.01 
df_original = reading_data(fraction)  
df          = df_original        
df_search   = df_original

In [179]:
def sortbyphi(df):
    '''Description:
    Sort each D_i increasingly accoording to phi
    '''
    z = np.sort(df['z'].unique())
    df['phi'] = np.arctan(df['x']/df['y'])
    
    modules = [] 
    
    for layer_i in z[::-1] :
        tmp = df.query(f'z=={layer_i}')
        tmp = tmp.sort_values('phi', ascending=True)
        
        tmp['used'] = False  # To accept or To Neglect

        modules.append(tmp)
    
    return modules

In [180]:
def findcandidatewindows(left_mod, mod, right_mod, dphi):
    '''Description: 
        Compute the first and last candidates(the window) according to acceptance range(dphi) for each hit.
        SUPPOSSING THAT ALL DATA ARE ORDERED ACCOURDING TO PHI. THIS PROCCESS WAS DONE Previously
        In case of add more information to the modules, one easily can add throught the iteration 
    '''
    
    # CONVENTION:     
    # l_m  m  r_m   the values are ordered.      
    #  |   |   |             
    #  |   |   |    phi up  
    #  |   |   |    phi      
    #  |   |   |    phi down 
    #  |   |   |          
    
    right_hit_max = [] 
    right_hit_min = [] 

    temporal = mod['phi'] 
    
    # ITERATION OVER PHI FOR RIGHT 
    
    for phi_i in mod['phi']: 
        #print("=")
        #print(phi_i)
        if str(phi_i) == 'nan' :     
            print(phi_i, "the value of phi_i is NaN ON RIGHT")
            m = "nan"               # minumum hit 
            M = "nan"               # maximum hit
            right_hit_min.append(m) 
            right_hit_max.append(M) 
            continue # 
        if str(phi_i) == 'NaN' :     
            print(phi_i, "the value of phi_i is NaN ON RIGHT")
            m = "nan"               # minumum hit 
            M = "nan"               # maximum hit
            left_hit_min.append(m) 
            left_hit_max.append(M) 
            continue # 
        # GET HIT 
        down      = phi_i - dphi 
        up        = phi_i + dphi 
        #print(down, up)
        
        condition = f'{down} <= phi <=  {up}'
        tmp_df = right_mod.query(condition)
        if not tmp_df.empty:
            m = tmp_df['hit_id'][tmp_df.index[0]]     # minumum hit 
            M = tmp_df['hit_id'][tmp_df.index[-1]]    # maximum hit 
            right_hit_min.append(m) 
            right_hit_max.append(M) 
        elif tmp_df.empty :

            m = "nan" #pd.np.nan                      # minumum hit 
            M = "nan" #pd.np.nan                      # maximum hit
            right_hit_min.append(m)  
            right_hit_max.append(M) 


            
    left_hit_max = [] 
    left_hit_min = [] 
    # ITERATION OVER PHI FOR LEFT
    for phi_i in mod['phi']:
        if str(phi_i) == 'NaN' :     
            print(phi_i, "the value of phi_i is NaN ON LEFT")
            m = "nan"               # minumum hit 
            M = "nan"               # maximum hit
            left_hit_min.append(m) 
            left_hit_max.append(M) 
            continue # 
        if str(phi_i) == 'nan' :     
            print(phi_i, "the value of phi_i is NaN ON left")
            m = "nan"               # minumum hit 
            M = "nan"               # maximum hit
            left_hit_min.append(m) 
            left_hit_max.append(M) 
            continue # 
        
        # GET HIT 
        down      = phi_i - dphi 
        up        = phi_i + dphi 
        condition = f'{down} <= phi <= {up}'
        tmp_df = left_mod.query(condition)
        #print("len LEFT", len(tmp_df))
        if not tmp_df.empty :
            m = tmp_df['hit_id'][tmp_df.index[0]]        # minumum hit 
            M = tmp_df['hit_id'][tmp_df.index[-1]]       # maximum hit  
            left_hit_min.append(m)
            left_hit_max.append(M)
        elif tmp_df.empty :
            print("data_frame is empty LEFT")
            m = "nan"               # minumum hit 
            M = "nan"               # maximum hit
            left_hit_min.append(m) 
            left_hit_max.append(M) 
            
    mod["right_hit_max"] = right_hit_max  
    mod["right_hit_min"] = right_hit_min  
    mod["left_hit_max"]  = left_hit_max   
    mod["left_hit_min"]  = left_hit_min   
    
    
    return mod

In [181]:
def trackseeding():
    global left_mod, mod, right_mod   
    '''Description: 
        Checks the preceding and following modules for compatible hits using the above results.
        
        All triplets in the search window are fitted and compared.
        
        and the best one is kept as a track seed.
        
        stores its best found triplet
        Finding triplets is ap- plied in first instance to the modules
        that are further apart from the collision point
        Each triplet is the seed of a forming track
    '''
    #Necessary functions.
    def fit(triplet): 
        phi_data = [ df.query(f'hit_id == {hit}')['phi'] for hit in triplet ]
        z_data   = [ df.query(f'hit_id == {hit}')['z'] for hit in triplet   ]
        phi_data = [ hit.values[0] for hit in phi_data                      ]                        
        z_data   = [ hit.values[0] for hit in z_data                        ]                    
        
        # Kind of fit: Linear
        fitting = np.polyfit(phi_data, z_data, 1)
        # IMPORTANT 
        # IMPORTANT 
        # REMEMBER TO PUT HERE THE VALUES OF SIGMA. 
        # IMPORTANT
        chiSquared = np.sum((np.polyval(fitting, z_data) - phi_data)**2) 
        return chiSquared    
    df_triplets = [] 
    for index, part in mod.iterrows():

        r_hit_max, r_hit_min = part["right_hit_max"], part["right_hit_min"]  
        l_hit_max, l_hit_min = part["left_hit_max"],  part["left_hit_min" ] 
        
        if r_hit_max is "nan":
            #print(r_hit_max)
            #print("pass NAN r_hit_max")
            continue 
        elif r_hit_min is "nan":
            #print(r_hit_min)
            #print("pass NAN r_hit_min")
            continue 
        elif l_hit_max is "nan":
            #print(l_hit_max)
            #print("pass NAN l_hit_max")
            continue 
        elif l_hit_min is "nan":
            #print(l_hit_min)
            #print("pass NAN l_hit_min")
            continue     
        r_phi_max = right_mod.query(f"hit_id == {r_hit_max}")['phi'].values[0]
        r_phi_min = right_mod.query(f"hit_id == {r_hit_min}")['phi'].values[0] 
        
        l_phi_max = left_mod.query(f"hit_id == {l_hit_max}")['phi'].values[0]  
        l_phi_min = left_mod.query(f"hit_id == {l_hit_min}")['phi'].values[0]  
        
        
        
        left_mod.query(f" {l_phi_min} <= phi <= {l_phi_max}")
        #Forming all Triplets. 
        # Here I am adding the condition of used and not used.
        # After of this I need to change the condition of used and not used. 
        #  
        # 
        #  
        # 
        #print("tmp_left")
        #print(tmp_left)
        #tmp_left = left_mod.query(f" {l_phi_min} <= phi <= {l_phi_max}")
        
        
        
        tmp_right = right_mod.query(f" {r_phi_min} <= phi <= {r_phi_max} & used == False  ")
        for R in tmp_right['phi'].values:
        #for L in tmp_left['phi'].values: 
            #tmp_right = right_mod.query(f" {r_phi_min} <= phi <= {r_phi_max}")
            #tmp_right = right_mod.query(f" {r_phi_min} <= phi <= {r_phi_max}")
            tmp_left = left_mod.query(f" {l_phi_min} <= phi <= {l_phi_max} & used == False ")
            #print("tmp_right")
            #print(tmp_right)
            #for R in tmp_right['phi'].values:
            #tmp_left = left_mod.query(f" {l_phi_min} <= phi <= {l_phi_max}")
            for L in tmp_left['phi'].values: 
                
                # I WILL SUPPOSE THAT 
                # All information it is found on the hit values.
                #print("VALUES  center left right ")
                hit_center = int(part["hit_id"])  
                #print(part["hit_id"])
                hit_left   = int(tmp_left.query(f" phi == {L}")['hit_id'].values[0] )    
                #print(tmp_left.query(f" phi == {L}")['hit_id'].values )
                hit_right  = int(tmp_right.query(f" phi == {R}")['hit_id'].values[0] )                          
                #print(tmp_right.query(f" phi == {R}")['hit_id'].values)
                # With this data we have built the triplets. 
                triplets = [hit_left, hit_center, hit_right]
                # This a lost of memory. I mean that call by hits and not by values is a lost 
                # of memory.
                #print("triplets")
                #print("triplets")
                #print("triplets")
                #print(triplets)
                chi2 = fit(triplets)                                                                                                                                                                
                # Finally we append the values of the data to a df_triplets
                
                #print("####?###")
                #print(triplets)
                #print(chi2)
                #print("#######")
                df_triplets.append(list(triplets)+[chi2])
    df_triplets = pd.DataFrame(df_triplets, columns = ['left_hit', 'hit', 'right_hit', 'chi2']) 
    #        print("=========")
    #        print(df_triplets)
    #        print("=========")    
    # Up to this point it is necessary to have the values of df_triplets complete
    # Then the algorithm should continue to get the best choices according to the values
    # of chi2.  
              
    def best_choice(df_triplets):
        seeds = []
        for hit_c in df_triplets['hit'].values :
            #print(hit_c)
            tmp = df_triplets.query(f'hit == {hit_c}')
            #print(tmp)
            minimum = (tmp['chi2']).idxmin()
            #i = chi2values.idxmin() # Here we choice the values.
            #tracks.append((tmp.loc[i]).values) 
            t = (tmp.loc[minimum]).values 
            t = [int(i) for i in t]
            
            
       
            ######################MARKING TRIPLES###########
            # MARKING EACH HIT AS USED ON THE WORKING MODULE  # LEFT
            hit_id_left = t[0] 
            index_left   = df.query(f"hit_id == {hit_id_left}").index[0]   #################### df works but it would be best to use
            left_mod.loc[index_left, "used"]   = True         #MARKING USED VALUE
            ################################################  # CENTER
            # MARKING EACH HIT AS USED ON THE WORKING MODULE 
            hit_id_center = t[1] 
            index_center = df.query(f"hit_id == {hit_id_center}").index[0]
            mod.loc[index_center, "used"]      = True            #MARKING USED VALUE
            ################################################  # RIGHT
            # MARKING EACH HIT AS USED ON THE WORKING MODULE
            hit_id_right = t[2] 
            index_right  = df.query(f"hit_id == {hit_id_right}").index[0]
            right_mod.loc[index_right, "used"] = True         #MARKING USED VALUE
            ################################################
            ###########################################################
            #these are the triplets
            #print(t[:3])
            #seeds.insert(0, list(t[:3]))
            seeds.append(list(t[:3]))    # Here I am negleting the information chi2 because is not important
            #print(t[:3])
        return seeds # obviously it is a track
    
    seeds = best_choice(df_triplets)
    return seeds 

In [182]:
def track_forwarding():
    global tracks, work_module, dphi # this step is important, because I'am using the values of work_module at the same time to modify.
    new_tracks = [] 
    # Notation:
    # x0, y0, z0 is the EXTRAPOLATED track.               
    # X,  Y,  Z  is the last track on previous module.   
    # x,  y,  z  is the tracks on a window.                                                                 
    # Searching tracks on phi_e - dphi < phi < d that minimize the extrapolated function.
    # r0 = np.array([x0, y0, z0] )
    # r  = np.array([x, y, 1] )
    # R  = np.array([X,  Y,  Z ] )
    def mod(r):
        return np.sqrt(np.sum(r*r))
    def ext_func(r0, r1, r):
        # r0, r1, r are arrays
        dx2_plus_dy2 = mod(r0-r)  # distance between hits on the working module.  
        dz2          = mod(r1-r0) # distance between the last two modules.                                  
        return dx2_plus_dy2/dz2   
    
    z_e = float(work_module['z'].unique()) #z_position of work_module
    #phi_e 
    #x_e 
    #y_e 
    #z_e 
    # here the track is exactly the seed. Only for the 1th iteration
    for track in tracks: 
        #PROOF: Do have the track values information of USED ?
        data = []          
        vector_data = []                                                                                                                    
        for hit in track[0:2] :                                       
            data.append(tuple((df.query(f'hit_id == {hit}')[['phi', 'z']]).values[0]))     
            vector_data.append(tuple((df.query(f'hit_id == {hit}')[['x', 'y', 'z']]).values[0]))
        phi_data, z_data = zip(*data) 
        #print(vector_data)
        #EXTRAPOLATED SEGMENT FUNCTION  
        ext_seg = interpolate.interp1d(z_data, phi_data, fill_value = "extrapolate" )
        phi_e = ext_seg(z_e)
        
        r_l, r_c = vector_data 
        r_l, r_c = np.array(r_l), np.array(r_c)
        
        x_e, y_e, z_e = r_e(z_e, r_l, r_c) # COMPUTING THE VALUES ON THE WORKING MODULE. It is good.
        
        #PROOF, HERE THE VALUE OF Z_E SHOULD BE THE SAME ON BOTH ********** IMPORTANT 
        
        #depend on z_e
        #y_e = #depend on z_e      
        
        #Open a Window centered on phi_e: 
        
        down = phi_e - dphi
        up   = phi_e + dphi   
        
        #################################### WINDOW ###########################
        df_work_module_window = work_module.query(f" {down} <= phi <= {up} " ) # I do not know how much time this line take 

        #This definition will be done after the loop.
        #df_candidates = pd.DataFrame(columns=["hit_id", "ext_fun"]) # This dataframe have to be reviwed
        hit_left = track[0]   
        R  = df.query(f'hit_id == {hit_left}')[['x','y','z']].values[0] # this value would have to change on each track
        r0 = np.array([x_e, y_e ,z_e ])

        #print(R, "this is R")
        #print(r0, "this is r0")
        tmp_candidates = []
        for index, row in df.iterrows(): # here the index is not important
            # Here I only need to have the information of position.
            r      =  row[['x', 'y', 'z']].values # this value is variable on each row.
            hit_id =  row['hit_id']    
            
            #print("PROOF")
            #print(r, hit_id)
            #Then I compute the value 
            ext_func_value = ext_func(r0, R, r)
            tmp_candidates.append( [hit_id, ext_func_value] )
            #print(tmp_candidates)
        
        if tmp_candidates == []:                                            
            print("an error ocurred: with tmp_candidates is empty ")                                          
            break 
        #print("=====")
        #print(tmp_candidates)
        #print("=====")
        df_candidates = pd.DataFrame(tmp_candidates, columns=["hit_id", "ext_fun"])
        # I will find the minimum value of the extrapolation function 
        #print("df_candidadtes ")
        #print(df_candidates['ext_fun'])
        #print("df_candidadtes")    
        index      = df_candidates['ext_fun'].idxmin()
        new_hit_id = df_candidates['hit_id'][index]
        new_hit_id = int(new_hit_id)
        
        # MARKING EACH HIT AS USED ON THE WORKING MODULE 
        
        work_module.loc[index, "used"] = True 
        new_track  = [new_hit_id] + track
        new_tracks.append(new_track)
    return new_tracks

# MAIN 

In [183]:
########################################################################################
################################# MAIN #################################################
########################################################################################
############################ GENERAL ALGORITHM #########################################
#####PARAMETERS#####
fraction = 0.004    
dphi     = 0.1  
m        = 22  # number of modules counted from the left.
               # from 1 to 24. No more. 
    
new_tracks = []                      # 
df = reading_data(fraction)          # where data is unmodified.
# df_search   = df_original          # where I am searching 
tracks = []                          # [[1,24, 5], [7,6,4] ,[346,7,32,], ... ]
# *********************IMPORTANT**********************************************
# The information of tracks is ordered. 
# Because each of its elements are an ordered list according to module layers.
# However, the information of hits are unique and not matter if are a ordered set. 
# But it was filled out in order



# SEPARATION BY MODULE
modules = sortbyphi(df) 
for M_i in range(len(modules)-1, len(modules)-m-1, -1):
#for M_i in range(22, 20, -1):
    t1 = time.time()
    print(f"module number {M_i}")
    M_i = M_i - 1
    #1th STEP: module #m-1 = M_i
    left_mod  =  modules[M_i - 1]  
    mod       =  modules[M_i    ]  
    right_mod =  modules[M_i + 1] 
    

    new_seeds   = trackseeding() 
    tracks      = tracks + new_seeds
    work_module = modules[M_i - 2]    
    new_tracks  = track_forwarding()

"""
    t2 = time.time()
    print("time per module", t2-t1) 
tracks = pd.Series(new_tracks)
print("FINDING TRACKS FINISHED ")
######################
######  FINALLY  #####
###### COMPARING #####
######################
df_real_tracks = df.groupby(['particle_id'])['hit_id'].unique() # this is a series.
#Scoring(df_real_tracks, tracks) 
######################
"""

module number 24


KeyError: 'right_hit_max'

In [ ]:
tracks         

In [ ]:
new_tracks     

In [ ]:
Scoring(df_real_tracks, new_tracks) 